**LCEL 메모리**
 LangChain Explicit Learning Memory의 약자로, LangChain에서 대화 내역을 효율적으로 관리하고 학습하는 데 사용되는 메모리 시스템 중 하나입니다. 이 메모리는 주로 **명시적 학습(Explicit Learning)** 을 기반으로 대화 데이터를 처리

* LCEL 메모리는 대화 중 발생한 정보를 명시적으로 저장하고, 이를 기반으로 사용자가 제공한 정보나 문맥을 학습합니다.
* 대화가 진행될수록 사용자가 언급한 데이터를 추적하고, 필요한 경우 이 데이터를 요약하거나 호출하여 적절한 응답을 생성합니다.

**ChatOpenAI 초기화**
* 결과의 결정적 특성을 높이기 위해 '온도=0.1'을 설정하여 OpenAI 모델을 초기화합니다.

**대화요약버퍼메모리**
* 대화 내용을 메모리를 통해 저장하고 요약합니다.
* 효율성을 보장하기 위해 토큰 크기를 제한합니다.

**ChatPromptTemplate**
* 대화의 구조를 정의합니다.
* 과거 대화 내용을 담고 사용자 질문을 처리하도록 설계되었습니다.

**load_memory 함수**
* 메모리에서 대화 기록을 로드합니다.
* 대화 기록을 체인에 전달합니다.

**체인 구성** 
* RunnablePassthrough를 사용하여 대화 기록을 "history" 변수에 전달합니다.
* 프롬프트와 모델을 결합하여 입력을 기반으로 응답을 생성합니다.

**invoke_chain 함수**
* 사용자의 질문을 입력으로 받아 체인을 실행합니다.
* 생성된 응답을 메모리에 저장합니다.
* 입력된 질문과 답변을 모두 저장합니다.
* 검증을 위해 결과를 인쇄합니다.

In [ ]:
# 필요한 LangChain 구성요소를 임포트
from langchain.memory import ConversationSummaryBufferMemory  # 대화 내역을 요약하여 관리하는 메모리 구성요소
from langchain.chat_models import ChatOpenAI  # OpenAI의 채팅 모델을 사용해 응답 생성
from langchain.schema.runnable import RunnablePassthrough  # 체인을 연결하기 위한 단순 실행기
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder  # 프롬프트 관련 유틸리티

# 낮은 온도로 OpenAI 채팅 모델 초기화 (결과를 더 일관되게 생성)
llm = ChatOpenAI(temperature=0.1)

# 대화 내용을 저장하고 요약하는 메모리 객체 정의
memory = ConversationSummaryBufferMemory(
    llm=llm,  # 요약을 위해 동일한 언어 모델 사용
    max_token_limit=120,  # 최대 토큰 크기를 120으로 제한
    return_messages=True,  # 요약 대신 대화 메시지 형식으로 반환
)

# 대화의 구조를 정의하는 프롬프트 객체 생성
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),  # 시스템 메시지로 AI의 역할을 정의
        MessagesPlaceholder(variable_name="history"),  # 이전 대화 기록을 가져올 자리 표시자
        ("human", "{question}"),  # 사용자의 질문이 들어갈 자리 표시자
    ]
)

# 메모리에서 대화 기록을 로드하는 함수 정의
def load_memory(_):
    return memory.load_memory_variables({})["history"]  # 메모리 변수에서 "history"를 반환

# 체인 정의
chain = (
    RunnablePassthrough.assign(history=load_memory)  # "history" 변수에 로드된 대화 기록을 전달
    | prompt  # 프롬프트와 결합
    | llm  # OpenAI 모델에 전달하여 응답 생성
)

# 체인을 실행하고 결과를 저장하는 함수 정의
def invoke_chain(question):
    # 체인을 실행하여 질문에 대한 응답 생성
    result = chain.invoke({"question": question})
    
    # 생성된 응답을 메모리에 저장
    memory.save_context(
        {"input": question},  # 사용자의 입력 저장
        {"output": result.content},  # 모델의 응답 저장
    )
    
    # 결과 출력
    print(result)


In [3]:
invoke_chain("My name is Mo")

content='Hello Mo! How can I assist you today?'


In [4]:
invoke_chain("What is my name?")

content='Your name is Mo.'
